In [2]:
import aiohttp
import asyncio
import pandas as pd
import random
import time
from datetime import datetime, timedelta, timezone
import logging

In [3]:
# === Настройки ===
header = {'User-Agent': 'HH-Data-Coll/v1.0 (contact 135861v@mail.ru)'}
area_id_russia = 113
per_page = 100
max_pages = 100
semaphore = asyncio.Semaphore(5)

# Опыт
experiences = ['noExperience', 'between1And3', 'between3And6', 'moreThan6']

# Настройки логгирования
log_filename = f"hh_parser_log_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.log"

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",  
    handlers=[
        logging.FileHandler(log_filename, encoding="utf-8"), 
        logging.StreamHandler()
    ]
)

def log_elapsed_time(start, task="Code"):
    end = time.perf_counter()
    elapsed = end - start
    mins, secs = divmod(elapsed, 60)
    hrs, mins = divmod(mins, 60)
    logging.info(f"⏱ Время выполнения ({task}): {int(hrs):02d}:{int(mins):02d}:{secs:.2f} секунд")


# Период: последние 30 дней с шагом 7 дней
def get_date_ranges(days=30, step=7):
    today = datetime.now(timezone.utc).date()
    start_date = today - timedelta(days=days)
    ranges = []
    current = start_date
    while current < today:
        end = min(current + timedelta(days=step), today)
        ranges.append((current.isoformat(), end.isoformat()))
        current = end
    return ranges

# Загрузка справочников для валидации данных
async def fetch_reference_data():
    async with aiohttp.ClientSession(headers=header) as session:
        async with session.get('https://api.hh.ru/dictionaries') as resp:
            dict_data = await resp.json()

        async with session.get('https://api.hh.ru/areas') as resp:
            area_data = await resp.json()
            area_ids = set()

            def extract_area_ids(areas):
                for a in areas:
                    area_ids.add(str(a['id']))
                    if 'areas' in a and a['areas']:
                        extract_area_ids(a['areas'])

            extract_area_ids(area_data)

        return {
            'experience': {e['name'] for e in dict_data['experience']},
            'employment': {e['name'] for e in dict_data['employment']},
            'schedule': {e['name'] for e in dict_data['schedule']},
            'area_ids': area_ids
        }

# RateLimiter
class RateLimiter:
    def __init__(self, max_rps=3, long_pause_every=500, long_pause_duration=60):
        self.max_rps = max_rps
        self.long_pause_every = long_pause_every
        self.long_pause_duration = long_pause_duration
        self.last_request = None
        self.request_count = 0
        self.lock = asyncio.Lock()

    async def wait(self):
        async with self.lock:
            now = time.monotonic()
            if self.last_request is not None:
                elapsed = now - self.last_request
                delay = max(0, 1.0 / self.max_rps - elapsed)
                if delay > 0:
                    await asyncio.sleep(delay)
            self.last_request = time.monotonic()
            self.request_count += 1
            if self.request_count % self.long_pause_every == 0:
                logging.info(f"Долгая пауза {self.long_pause_duration} сек после {self.request_count} запросов...")
                await asyncio.sleep(self.long_pause_duration)

limiter = RateLimiter()


In [4]:
#=== Функции сбора вакансий ===
# Получение IT-ролей
async def fetch_it_roles():
    url = 'https://api.hh.ru/professional_roles'
    async with aiohttp.ClientSession(headers=header) as session:
        async with session.get(url) as response:
            response.raise_for_status()
            data = await response.json()
            for cat in data['categories']:
                if int(cat['id']) == 11:
                    roles = [
                        {'id': int(role['id']), 'name': role['name']}
                        for role in cat['roles']
                        if int(role['id']) not in [12, 25, 34, 155]
                    ]
                    logging.info(f'🔹 Найдено {len(roles)} IT-ролей')
                    return roles
            return []

# Загрузка одной страницы с повторными попытками
async def fetch_page(session, params, page, desc, max_retries=3):
    params['page'] = page
    for attempt in range(1, max_retries + 1):
        await limiter.wait()
        try:
            async with semaphore:
                async with session.get('https://api.hh.ru/vacancies', params=params) as response:
                    if response.status == 403:
                        logging.warning(f"🚫 Ошибка 403 на странице {page} — {desc} (попытка {attempt})")
                    elif response.status >= 400:
                        logging.warning(f"⚠️ Ошибка {response.status} на странице {page} — {desc} (попытка {attempt})")
                    response.raise_for_status()
                    return await response.json()
        except Exception as e:
            logging.warning(f"⚠️ Исключение при загрузке страницы {page} — {desc} (попытка {attempt}): {e}")
            await asyncio.sleep(random.uniform(1.0, 2.0))

    error_msg = f"❌ Ошибка: не удалось загрузить страницу {page} — {desc} после {max_retries} попыток. Остановка."
    logging.error(error_msg)
    raise RuntimeError(error_msg)

# Сбор вакансий по комбинации ролей, опыта и дат
async def fetch_vacancies(session, role_id, role_name, experience, date_from, date_to):
    desc = f"роль '{role_name}', опыт '{experience}', {date_from}–{date_to}"
    vacancies = []
    params = {
        'professional_role': role_id,
        'area': area_id_russia,
        'experience': experience,
        'date_from': date_from,
        'date_to': date_to,
        'per_page': per_page,
    }

    for page in range(max_pages):
        data = await fetch_page(session, params, page, desc)
        if data is None:
            break
        items = data.get('items', [])
        if not items:
            break
        vacancies.extend(items)
        if page >= data.get('pages', 0) - 1:
            break
        await asyncio.sleep(random.uniform(0.3, 0.6))

    logging.info(f"✅ Собрано {len(vacancies)} вакансий — {desc}")
    return vacancies

# Основной сбор данных
async def collect_vacancies_async():
    all_vacancies = []
    date_ranges = get_date_ranges()
    roles = await fetch_it_roles()

    try:
        async with aiohttp.ClientSession(headers=header) as session:
            for role in roles:
                logging.info(f"\n== Роль: {role['name']} (ID {role['id']}) ==")
                for exp in experiences:
                    for date_from, date_to in date_ranges:
                        batch = await fetch_vacancies(session, role['id'], role['name'], exp, date_from, date_to)
                        all_vacancies.extend(batch)

    except RuntimeError as critical_error:
        logging.critical(f"🚨 Парсинг прерван: {critical_error}")
        raise  

    # logging.info(f"\n🚀 Всего собрано {len(all_vacancies)} вакансий")
    return all_vacancies

# Преобразование вакансий в DataFrame
def extract_vacancy_data(raw_vacancies):
    extracted = []
    for v in raw_vacancies:
        area_info = v.get('area', {})
        salary_range = v.get('salary_range') or {}
        
        extracted.append({
            'load_datetime': datetime.now(),
            'professional_role': v.get('professional_roles', [{}])[0].get('name') 
                                 if v.get('professional_roles') else None,
            'id': v.get('id'),
            'name': v.get('name'),
            'area': area_info.get('name'),
            'area_id': area_info.get('id'),
            'employer': v.get('employer', {}).get('name'),
            'accredited_it_employer': v.get('employer', {}).get('accredited_it_employer'),
            'trusted_employer': v.get('employer', {}).get('trusted'),
            'published_at': v.get('published_at'),
            'created_at': v.get('created_at'),
            'closed_at': v.get('closed_at'),
            'archived': v.get('archived'),
            'url': v.get('alternate_url'),
            'salary_from': v.get('salary', {}).get('from') if v.get('salary') else None,
            'salary_to': v.get('salary', {}).get('to') if v.get('salary') else None,
            'currency': v.get('salary', {}).get('currency') if v.get('salary') else None,
            'salary_mode': (salary_range.get('mode') or {}).get('name'),        # За месяц, за час...
            'salary_frequency': (salary_range.get('frequency') or {}).get('name'),  # Раз в месяц и т.п.
            'experience': v.get('experience', {}).get('name'),
            'schedule': v.get('schedule', {}).get('name'),
            'employment': v.get('employment', {}).get('name'),
        })
    return pd.DataFrame(extracted).drop_duplicates('id', keep='last')


# Валидация данных
def validate_vacancies(df: pd.DataFrame, dicts: dict):
    required_fields = ['id', 'name', 'area', 'area_id', 'published_at']
    errors = []

    def is_empty(val):
        return val is None or val == "" or (isinstance(val, list) and len(val) == 0)

    invalid_rows = []

    for i, row in df.iterrows():
        row_errors = []

        # Проверка обязательных полей на пустоту
        for field in required_fields:
            if field not in row or is_empty(row[field]):
                row_errors.append(f"Пустое обязательное поле '{field}'")

        # Проверка значений из справочников
        if 'experience' in row and row['experience'] not in dicts['experience'] and not is_empty(row['experience']):
            row_errors.append(f"Неизвестное experience: {row['experience']}")
        if 'employment' in row and row['employment'] not in dicts['employment'] and not is_empty(row['employment']):
            row_errors.append(f"Неизвестное employment: {row['employment']}")
        if 'schedule' in row and row['schedule'] not in dicts['schedule'] and not is_empty(row['schedule']):
            row_errors.append(f"Неизвестное schedule: {row['schedule']}")
        if 'area_id' in row and str(row['area_id']) not in dicts['area_ids'] and not is_empty(row['area_id']):
            row_errors.append(f"Неизвестный area_id: {row['area_id']}")

        if row_errors:
            errors.append((i, row_errors))
            invalid_rows.append(i)

    if errors:
        for i, errs in errors:
            logging.warning(f"Строка {i} отклонена: {', '.join(errs)}")

    reject_df = df.loc[invalid_rows].copy()
    valid_df = df.drop(index=invalid_rows).reset_index(drop=True)

    return valid_df.reset_index(drop=True), reject_df.reset_index(drop=True)



In [ ]:
# Запуск сбора вакансий
start = time.perf_counter()

raw_vacancies = await collect_vacancies_async()
df = extract_vacancy_data(raw_vacancies)

reference_data = await fetch_reference_data()
valid_df, reject_df = validate_vacancies(df, reference_data)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
reject_filename = f"rejected_vacancies_{timestamp}_reject.csv"
if not reject_df.empty:
    reject_df.to_csv(reject_filename, index=False)
    logging.info(f"\n🚫 {len(reject_df)} вакансий отклонено и сохранено в файл {reject_filename}")
else:
    logging.info("\n✅ Все вакансии прошли валидацию. Reject-файл не создан.")


logging.info(f"\n🚀 Всего собрано вакансий: {len(raw_vacancies)}")
logging.info(f"📦 Загружено уникальных id: {valid_df['id'].nunique()}")

log_elapsed_time(start, "hh_parser")


In [6]:
valid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47519 entries, 0 to 47518
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   load_datetime           47519 non-null  datetime64[ns]
 1   professional_role       47519 non-null  object        
 2   id                      47519 non-null  object        
 3   name                    47519 non-null  object        
 4   area                    47519 non-null  object        
 5   area_id                 47519 non-null  object        
 6   employer                47519 non-null  object        
 7   accredited_it_employer  47394 non-null  object        
 8   trusted_employer        47519 non-null  bool          
 9   published_at            47519 non-null  object        
 10  created_at              47519 non-null  object        
 11  closed_at               0 non-null      object        
 12  archived                47519 non-null  bool  

In [10]:
valid_df.head()

,load_datetime,professional_role,id,name,area,area_id,employer,accredited_it_employer,trusted_employer,published_at,...,archived,url,salary_from,salary_to,currency,salary_mode,salary_frequency,experience,schedule,employment
0,2025-08-14 19:09:30.423965,"BI-аналитик, аналитик данных",123077313,Младший аналитик данных,Казань,88,Магнетто,True,True,2025-07-21T23:41:46+0300,...,False,https://hh.ru/vacancy/123077313,NaN,NaN,None,None,None,Нет опыта,Полный день,Полная занятость
1,2025-08-14 19:09:30.424234,"BI-аналитик, аналитик данных",121753731,Аналитик Power BI (стажер),Москва,1,Ельмикеев Евгений Иванович,False,True,2025-07-17T17:44:40+0300,...,False,https://hh.ru/vacancy/121753731,35000.0,NaN,RUR,За месяц,Два раза в месяц,Нет опыта,Удаленная работа,Полная занятость
2,2025-08-14 19:09:30.424332,"BI-аналитик, аналитик данных",123065130,Data Steward (стажер),Москва,1,Московская Биржа,False,True,2025-07-21T15:44:46+0300,...,False,https://hh.ru/vacancy/123065130,NaN,NaN,None,None,None,Нет опыта,Полный день,Частичная занятость
3,2025-08-14 19:09:30.424369,"BI-аналитик, аналитик данных",122741277,Специалист по информационным технологиям (госс...,Казань,88,Республика Татарстан,False,True,2025-07-16T09:53:41+0300,...,False,https://hh.ru/vacancy/122741277,47000.0,57000.0,RUR,За месяц,None,Нет опыта,Полный день,Полная занятость
4,2025-08-14 19:09:30.424412,"BI-аналитик, аналитик данных",122841723,Аналитик данных,Ангарск,1126,"Командор, сеть супермаркетов",False,True,2025-07-16T06:43:25+0300,...,False,https://hh.ru/vacancy/122841723,41000.0,NaN,RUR,За месяц,Два раза в месяц,Нет опыта,Полный день,Полная занятость


In [ ]:
# Сбор key_skills - черновик

# async def extract_key_skills(raw_vacancies: list, session: aiohttp.ClientSession, rate_limiter, logger) -> pd.DataFrame:
#     all_key_skills = []
#     total = len(raw_vacancies)

#     for idx, vac in enumerate(raw_vacancies, 1):
#         vac_id = vac.get("id")
#         if not vac_id:
#             continue

#         url = f"https://api.hh.ru/vacancies/{vac_id}"

#         await rate_limiter.wait()

#         try:
#             async with session.get(url) as response:
#                 if response.status != 200:
#                     logger.warning(f"[{idx}/{total}] Не удалось получить key_skills для вакансии {vac_id}, статус {response.status}")
#                     all_key_skills.append({"vacancy_id": vac_id, "key_skill": "не указаны"})
#                     continue
#                 vacancy_detail = await response.json()
#         except Exception as e:
#             logger.warning(f"[{idx}/{total}] Ошибка при запросе key_skills для {vac_id}: {e}")
#             all_key_skills.append({"vacancy_id": vac_id, "key_skill": "не указаны"})
#             continue

#         key_skills = vacancy_detail.get("key_skills", [])
#         if not key_skills:
#             all_key_skills.append({"vacancy_id": vac_id, "key_skill": "не указаны"})
#         else:
#             for skill_entry in key_skills:
#                 skill = skill_entry.get("name", "").strip()
#                 all_key_skills.append({"vacancy_id": vac_id, "key_skill": skill})

#         if idx % 500 == 0 or idx == total:
#             logger.info(f"🔹 Обработано {idx}/{total} вакансий для key_skills")

#     return pd.DataFrame(all_key_skills)


In [ ]:
# # Сбор вакансий и key_skills - черновик

# start = time.perf_counter()
# raw_vacancies = await collect_vacancies_async()
# df = extract_vacancy_data(raw_vacancies)

# # Валидация вакансий
# reference_data = await fetch_reference_data()
# valid_df, reject_df = validate_vacancies(df, reference_data)

# # Загрузка key_skills
# async with aiohttp.ClientSession(headers=header) as session:
#     df_key_skills = await extract_key_skills(raw_vacancies, session, limiter, logging)

# # Информация по результатам
# logging.info(f"\n🚀 Всего собрано вакансий: {len(raw_vacancies)}")
# logging.info(f"📦 Загружено уникальных id: {valid_df['id'].nunique()}")
# logging.info(f"🔑 Сформировано строк в key_skills: {len(df_key_skills)}")

# log_elapsed_time(start, "hh_parser + key_skills")
